In [1]:
import rosbag
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cv2
from cv_bridge import CvBridge
import ipyvolume as ipv
from decimal import Decimal


#from zed_interfaces.msg import ObjectsStamped


In [2]:
#READ BAG

fileroot='24_06_24/'
filename='2024-06-24-BT-Maleen-02'

saverpath='/home/maleen/rosbags/research_data/Transformers/datasets/body_tracking_data/'

bag  = rosbag.Bag('/home/maleen/research_data/Transformers/datasets/'+fileroot+ filename + '.bag')

In [15]:
# Initialize variables
skeleton_size = 0
image_size = 0

skeleton_timestamps = []
skeleton_3d_data = []

image_timestamps = np.array([])
image_data = np.array([])

topiclist = ['/body_tracker/image', '/visualization_marker_array']


for topic, msg, t in bag.read_messages(topics=topiclist):
    if topic == '/visualization_marker_array':
            # Extract the timestamp from the first marker
        timestamp = (msg.markers[0].header.stamp.secs) + (Decimal(msg.markers[0].header.stamp.nsecs) / 1000000000)
        skeleton_timestamps.append(timestamp)

        print((msg.markers))

        break
    
            
            #skeleton_3d_data.append(current_markers)

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [13]:
np.shape(skeleton_timestamps)

(608,)

In [4]:
# Convert the list to a numpy array
skeleton_3d_data = np.array(skeleton_3d_data)

# Print to check
print("Skeleton Timestamps: ", skeleton_timestamps)
print("Skeleton 3D Data: ", skeleton_3d_data)

Skeleton Timestamps:  [Decimal('1719201762.308698654'), Decimal('1719201762.308698654'), Decimal('1719201762.308698654'), Decimal('1719201762.308698654'), Decimal('1719201762.308698654'), Decimal('1719201762.308698654'), Decimal('1719201762.308698654'), Decimal('1719201762.308698654'), Decimal('1719201762.308698654'), Decimal('1719201762.308698654'), Decimal('1719201762.308698654'), Decimal('1719201762.308698654'), Decimal('1719201762.308698654'), Decimal('1719201762.308698654'), Decimal('1719201762.308698654'), Decimal('1719201762.308698654'), Decimal('1719201762.308698654'), Decimal('1719201762.308698654'), Decimal('1719201762.308698654'), Decimal('1719201762.308698654'), Decimal('1719201762.308698654'), Decimal('1719201762.308698654'), Decimal('1719201762.308698654'), Decimal('1719201762.308698654'), Decimal('1719201762.308698654'), Decimal('1719201762.308698654'), Decimal('1719201762.308698654'), Decimal('1719201762.308698654'), Decimal('1719201762.308698654'), Decimal('1719201762.

In [12]:
skeleton_3d_data[2]

array([[ 0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ],
       [ 0.8942519 , -0.12707821,  1.235     ],
       [ 0.76672447, -0.13166741,  1.436     ],
       [ 0.62376492,  0.11292299,  1.32      ],
       [ 0.        ,  0.        ,  0.        ]])

In [6]:
bridge = CvBridge()
detected_images=[]

for img in image_data:
    
    cv_image = bridge.imgmsg_to_cv2(img, desired_encoding="passthrough")
    detected_images.append(cv_image)

#TEST DETECTED IMAGES



plt.imshow(cv2.cvtColor(detected_images[300], cv2.COLOR_BGR2RGB))
#plt.imshow(detected_images[25])
plt.show()

IndexError: list index out of range

In [ ]:

# Original resolution
old_width, old_height = 1280, 720

# New resolution
new_width, new_height = 640, 360

# Calculate scaling factors
scale_x = new_width / old_width
scale_y = new_height / old_height

# Initialize a list to store scaled skeletal data
scaled_skeletal_data = []

# Iterate over each frame/observation
for frame in skeleton_2d_data:
    # Scale each point in the frame
    scaled_frame = [[x * scale_x, y * scale_y] if x > 0 and y > 0 else [x, y] for x, y in frame]
    scaled_skeletal_data.append(scaled_frame)

# Convert the list back to a NumPy array if needed
scaled_skeletal_data = np.array(scaled_skeletal_data, dtype=np.float32)

# Now, scaled_skeletal_data contains the scaled skeletal points for each frame/observation


In [ ]:
test_img=detected_images[300]

for point in scaled_skeletal_data[300]:
    x, y = int(point[0]), int(point[1])
    #print(x)
    cv2.circle(test_img, (x, y), radius=3, color=(255, 255, 0), thickness=1)  # Draw a green circle at each point
    
plt.imshow(cv2.cvtColor(test_img, cv2.COLOR_BGR2RGB))
#plt.imshow(detected_images[25])
plt.show()

In [ ]:
# Assuming `skeleton_3d_data[500]` is your input data
data = np.array(skeleton_3d_data[300])

# Filter out the NaN values from the first 18 keypoints
valid_keypoints = ~np.isnan(data[:18, :].any(axis=1))
filtered_data = data[:18][valid_keypoints]


# Define connections based on your body model (this is a basic example)
original_connections = [
    (16, 14),(14, 0),
    (17, 15),(15, 0),
    (0, 1),
    (1, 2), (2, 3), (3, 4),          # Right arm
    (1, 5), (5, 6), (6, 7),          # Left arm
    (1, 8), (8, 9), (9, 10),        # Right leg
    (1, 11), (11, 12), (12, 13),    # Left leg
    (8, 11)                          # Between hips
]

# Create a mapping from old indices to new indices after NaN removal
index_mapping = {old_index: new_index for new_index, old_index in enumerate(np.where(valid_keypoints)[0])}

# Create a new list of connections that only includes the valid keypoints
new_connections = [(index_mapping[start], index_mapping[end])
                   for start, end in original_connections
                   if start in index_mapping and end in index_mapping]

# Create the 3D scatter plot
ipv.figure()
scatter = ipv.scatter(filtered_data[:, 0], filtered_data[:, 1], filtered_data[:, 2], marker='sphere', size=2)

# Draw lines for the new skeletal links
for start, end in new_connections:
    ipv.plot(filtered_data[[start, end], 0], filtered_data[[start, end], 1], filtered_data[[start, end], 2], color='red')

# Show the plot
ipv.view(azimuth=0, elevation=-90)
ipv.show()

In [ ]:
n = 34# number of rows to remove from the front
m = 30 # number of rows to remove from the end

# Remove n rows from the front and m rows from the end
df_trimmed = df.iloc[n:-m]

# Reset index and remove old index column
df_trimmed = df_trimmed.reset_index(drop=True)

remainder = df.shape[0] % 60
remainder

df_trimmed.shape

In [ ]:
df_trimmed.to_pickle(saverpath + filename + '.pkl')

In [ ]:
import datetime

# Example timestamp in nanoseconds
timestamp_nano = 1709874089634082714

# Convert nanoseconds to seconds
timestamp_sec = timestamp_nano / 1e9

# Create datetime object
timestamp = datetime.datetime.fromtimestamp(timestamp_sec)

print(timestamp)

In [ ]:
df_trimmed

In [ ]:
df_trimmed